# Process model prediction

In [81]:
import glob
import numpy as np
import pandas as pd 
import xarray as xr

import tensorflow as tf

## Open dataset for prediction

In [86]:
path_file = "../data/validate/2019_04_ERA5.nc"
ds = xr.open_dataset(path_file)

We are going to manually recreate the functionality of the ```slice_generator``` class here,
just to pull out an input for the model.

In [87]:
# Choose arbitrary slice
start = 427
end   = 429

In [88]:
# Extract slice and reshape
array = ds['t2m'].isel(time=slice(start, end)).values
print(array.shape)
array = np.moveaxis(array, 0, 1)
print(array.shape)
array = array.reshape(-1, 2, 1, 21, 21)
print(array.shape)

(2, 21, 21)
(21, 2, 21)
(1, 2, 1, 21, 21)


## Import model and predict

In [103]:
model_dir = "../models/"
models_list = sorted(glob.glob(model_dir+"*.h5"))
print(models_list)

['../models/full_stack_1f_1c_21x_21y.h5', '../models/full_stack_1f_1c_21x_21y_1.h5', '../models/t_full_stack_1f_1c_21x_21y.h5']


In [104]:
# choose a model
file_index = 
models_list[file_index]

model = tf.keras.models.load_model(models_list[file_index])
# model.summary()

In [106]:
model.history

Make prediction here:

In [100]:
pred = model.predict(array, verbose=1)
#pred

1/1 [==============================] - 0s 2ms/step


array([[[[[286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908],
          [286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908],
          [286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908],
          [286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
           286.77908, 286.77908, 286.77908, 286.77908, 286.77908,
       

## Convert prediction back to ```netcdf``` file

Reshape model output

In [92]:
print(pred.shape)
out = pred.reshape(1,21,21)
print(out.shape)

(1, 1, 1, 21, 21)
(1, 21, 21)


Get timestamp of prediction

In [93]:
pred_time = ds['t2m'].isel(time=slice(end-1, end))['time'].values[0]
ts = pd.to_datetime(str(pred_time)) 
d = ts.strftime('%Y_%m_%d_%H')
d

'2019_04_18_20'

 Create new xarray DataSet with same dimensions and coordinates as original, and save it as a ```netcdf``` file in ```../data/pred/```

In [94]:
data = xr.DataArray(data = out,
                     dims=('time', 'latitude', 'longitude',),
                     coords={'latitude': ds['latitude'].values,
                            'longitude': ds['longitude'].values,
                            'time': ds['t2m'].isel(time=slice(end-1, end))['time'].values,
                            })
data = xr.Dataset(data_vars={'t2m':data})
data.to_netcdf("../data/pred/"+d+"pred.nc")
data

<xarray.Dataset>
Dimensions:    (latitude: 21, longitude: 21, time: 1)
Coordinates:
  * latitude   (latitude) float32 40.0 39.75 39.5 39.25 ... 35.5 35.25 35.0
  * longitude  (longitude) float32 -125.0 -124.75 -124.5 ... -120.25 -120.0
  * time       (time) datetime64[ns] 2019-04-18T20:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 286.77908 ... 286.77908